## 4. Model Optimization

In [ ]:
def optimize_model(model, model_name):
    """Optimize model using TensorFlow's built-in capabilities"""
    print(f"\nOptimizing {model_name}...")
    
    # Create models directory
    os.makedirs('models', exist_ok=True)
    base_path = f'models/{model_name}'
    
    # Save original model
    model.save(f'{base_path}_original.h5')
    original_size = os.path.getsize(f'{base_path}_original.h5') / 1024  # KB
    
    try:
        # Configure TFLite converter
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        
        # Enable optimizations
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        
        # Enable float16 quantization
        converter.target_spec.supported_types = [tf.float16]
        
        # Add support for TF ops
        converter.target_spec.supported_ops = [
            tf.lite.OpsSet.TFLITE_BUILTINS,
            tf.lite.OpsSet.SELECT_TF_OPS
        ]
        
        # Convert model
        print("Converting to TFLite format...")
        quantized_model = converter.convert()
        
        # Save quantized model
        quantized_path = f'{base_path}_quantized.tflite'
        with open(quantized_path, 'wb') as f:
            f.write(quantized_model)
            
        # Compare model sizes
        sizes = {
            'Original': original_size,
            'Quantized': os.path.getsize(quantized_path) / 1024
        }
        
        # Plot size comparison
        plt.figure(figsize=(8, 5))
        plt.bar(sizes.keys(), sizes.values())
        plt.title('Model Size Comparison')
        plt.ylabel('Size (KB)')
        plt.xticks(rotation=45)
        
        for i, v in enumerate(sizes.values()):
            plt.text(i, v + 1, f'{v:.1f} KB', ha='center')
        
        plt.tight_layout()
        plt.show()
        
        # Print size comparison
        print("\nModel Size Comparison:")
        for name, size in sizes.items():
            print(f"{name}: {size:.2f} KB")
        print(f"Size reduction: {((original_size - sizes['Quantized']) / original_size * 100):.2f}%")
        
        return True, quantized_path
        
    except Exception as e:
        print(f"\nError during optimization: {str(e)}")
        print("Falling back to original model")
        return False, f'{base_path}_original.h5'

# Get best performing model
best_model_name = max(results.items(), key=lambda x: x[1]['accuracy'])[0]
best_model = results[best_model_name]['model']
print(f"Best performing model: {best_model_name}")

# Optimize the model
success, model_path = optimize_model(best_model, f"best_model_{best_model_name}")

if success:
    print(f"\nOptimized model saved to: {model_path}")
else:
    print(f"\nOriginal model saved to: {model_path}")